In [83]:
import numpy as np
import pandas as pd

In [84]:
#Meses invierno verano, 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos')

inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes', value_name='inv_verano')
inv_verano = inv_verano.sort_values(by=['mes']).reset_index(drop=True)

# inv_verano = pd.pivot_table(data=inv_verano, index=['zona'], columns=['mes'], values='inv_verano')
inv_verano.head()

,zona,mes,inv_verano
0,valdivia,abril,0
1,arauco,abril,0
2,constitucion,abril,1
3,chillan,abril,0
4,valdivia,agosto,0


In [85]:
#Carga de plantas

base_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='plantas')
display(base_plantas.head())

#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
display(parametros_planta.head())

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
display(consumo_plantas.head())


,nombre_planta,producto
0,planta1,EUNI SC
1,planta1,EUNI CC
2,planta1,EUGL SC
3,planta1,EUGL CC


,nombre_planta,producto,capacidad,stock_seguridad_planta,stock_inicial
0,planta1,EUNI SC,40,20,20
1,planta1,EUNI CC,40,20,20
2,planta1,EUGL SC,40,20,20
3,planta1,EUGL CC,40,20,20


,nombre_planta,producto,mes_planificado,volumen_consumo
0,planta1,EUNI SC,1,100
1,planta1,EUNI SC,2,100
2,planta1,EUNI SC,3,100
3,planta1,EUGL SC,1,100
4,planta1,EUGL SC,2,100


In [86]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
base_ingresos.head()


,id_ingreso,nombre_ingreso,nombre_planta,producto,zona,costo existencia,costo transporte,tipo_ingreso,almacen_verano,almacen_invierno,tipo_almacen,cosecha_compra
0,1,produccion EUNI SC constitucion,planta1,EUNI SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
1,2,produccion EUNI arauco sc,planta1,EUNI SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha
2,3,produccion EUNI valdivia sc,planta1,EUNI SC,valdivia,10,20,directo_planta,N_A,N_A,bosque,cosecha
3,4,produccion EUGL constitucion sc,planta1,EUGL SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
4,5,produccion EUGL arauco sc,planta1,EUGL SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha


In [87]:
#Carga Ingreso: Volumen mensual
base_ingreso_vol = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen')
base_ingreso_vol.head()

,id_ingreso,mes,volumen
0,1,1,20
1,1,2,30
2,1,3,40
3,2,1,20
4,2,2,30


In [88]:
#Carga de Almacenes

base_almacenes = pd.read_excel('../02_data/Almacenes.xlsx', sheet_name='almacenes')
base_almacenes.head()

,id_almacen,nombre_almacen,nombre_planta,producto,zona,id_almacen_acanche,carpeta,cosecha_compra,tipo_almacen,costo_existencia,capacidad
0,alm1,Cancha compra EUNI,planta1,EUNI CC,chillan,N_A,N_A,compra,cancha,10,100
1,alm2,Cancha compra sc EUNI,planta1,EUNI SC,chillan,N_A,N_A,compra,cancha,10,100
2,alm3,Cancha compra EUGL,planta1,EUGL CC,chillan,N_A,N_A,compra,cancha,10,100
3,alm4,Cancha compra sc EUGL,planta1,EUGL SC,chillan,N_A,N_A,compra,cancha,10,100
4,alm5,Cancha cosecha EUGL sc,planta1,EUGL SC,chillan,N_A,N_A,cosecha,cancha,10,100


In [89]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

In [92]:
## Matriz de demandas.
proyeccion = 12

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_seguridad_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'rango', 'demanda_rango']].dropna()
df

,nombre_planta,producto,mes_planificado,rango,demanda_rango
9,planta1,EUGL CC,1,0-3,70.0
10,planta1,EUGL CC,1,4-6,20.0
11,planta1,EUGL CC,1,7-9,10.0
12,planta1,EUGL CC,2,0-3,84.0
13,planta1,EUGL CC,2,4-6,12.0
14,planta1,EUGL CC,2,7-13,24.0
15,planta1,EUGL CC,3,0-3,72.0
16,planta1,EUGL CC,3,4-6,24.0
17,planta1,EUGL CC,3,7-9,12.0
18,planta1,EUGL CC,3,10-13,12.0
